** Initial Steps to be Followed to create bucket**
* Importing all necesssary libraries
* Creating S3 bucket (could also be created from the UI)
* Mapping train and test data in S3
* Mapping the path of models in S3

In [2]:
import sagemaker  # for built in algo download
import boto3      # reading s3 bucket(more like pandas)
from sagemaker.amazon.amazon_estimator import get_image_uri # 
from sagemaker.session import s3_input, Session

In [18]:
bucket_name = 'mytestbucket-bankapplication' # unique name for your bucket
my_region = boto3.session.Session().region_name
print(my_region)

eu-central-1


In [19]:
s3 = boto3.resource('s3')
try:
    if my_region == 'eu-central-1':
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
        print('S3 bucket created yeeeeehhh..!!')
except Exception as e:
    print('s3 bucket creation error..!!', e)

S3 bucket created yeeeeehhh..!!


In [20]:
# set output path to s3 or in s3 bucket 
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://mytestbucket-bankapplication/xgboost-as-a-built-in-algo/output


***Now that we've created the bucket, let's download the dataset from the web and store it in our new S3 bucket "mytestbucket-bankapplication."***

In [22]:
import pandas as pd
import urllib.request

# Download bank dataset from web
url = "https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv"
try:
    urllib.request.urlretrieve(url, "bank_clean.csv")
    print('Dataset downloaded successfully.')
except Exception as e:
    print('Error downloading dataset:', e)

# Load the downloaded dataset into a Pandas DataFrame
try:
    model_data = pd.read_csv('./bank_clean.csv', index_col=0)
    print('Dataset loaded successfully.')
except Exception as e:
    print('Error loading dataset:', e)


Dataset downloaded successfully.
Dataset loaded successfully.


In [30]:
model_data.head(5)

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0


In [23]:
import numpy as np

# Shuffle the data and split into train and test sets 70 train and 30 test
shuffled_data = model_data.sample(frac=1, random_state=1729)
train_size = int(0.7 * len(shuffled_data))
train_data, test_data = shuffled_data[:train_size], shuffled_data[train_size:]

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)


Train data shape: (28831, 61)
Test data shape: (12357, 61)


In [24]:
import os


# Concatenate train_data and save to CSV
train_data_concatenated = pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1)
train_data_concatenated.to_csv('train.csv', index=False, header=False)

# Upload train.csv to S3
s3_client = boto3.client('s3')
s3_client.upload_file('train.csv', bucket_name, os.path.join(prefix, 'train/train.csv'))

# Create s3_input object for training
s3_input_train = s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')


The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


**** In SageMaker, it's a convention to have the dependent variable as the first column in the dataset. Therefore, the "y_yes" column should be placed as the first column in the dataset.****

In [26]:
# Concatenate test_data and save to CSV
test_data_concatenated = pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1)
test_data_concatenated.to_csv('test.csv', index=False, header=False)

# Upload test.csv to S3
s3_client = boto3.client('s3')
s3_client.upload_file('test.csv', bucket_name, os.path.join(prefix, 'test/test.csv'))

# Create s3_input object for testing
s3_input_test = s3_input(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')


The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


**Building and Training Model Xgboost- Inbuilt Algorithm (already present in sagemaker)** 

In [32]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [33]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [35]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [36]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})


INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-03-07-15-12-30-768


2024-03-07 15:12:30 Starting - Starting the training job...
2024-03-07 15:12:46 Starting - Preparing the instances for training.........
2024-03-07 15:14:10 Downloading - Downloading input data...
2024-03-07 15:14:56 Training - Training image download completed. Training in progress...[2024-03-07 15:15:12.593 ip-10-0-190-68.eu-central-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single n

In [38]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type= 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-03-07-15-21-46-568
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-03-07-15-21-46-568
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-03-07-15-21-46-568


------!

In [40]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = CSVSerializer()
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')
predictions_array = np.fromstring(predictions[1:], sep=',')
print(predictions_array.shape)


(12357,)


In [41]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [42]:
# Confusion Matrix (ref: AWS documentation)

cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



**Result :** 
    * The matrix shows the model's performance in predicting "No Purchase" and "Purchase" outcomes. It correctly predicts "No Purchase" 91% of the time and "Purchase" 66% of the time, with an overall accuracy rate of 89.7%.

**After training our model and experimenting with SageMaker, it's important to delete all resources to prevent additional charges on the credit card.**

In [ ]:
# Delete the endpoint associated with the XGBoost predictor
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)

# Get a reference to the S3 bucket to be deleted
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)

# Delete all objects (files) in the S3 bucket
bucket_to_delete.objects.all().delete()

# Finally, delete the empty S3 bucket
s3.delete_bucket(Bucket=bucket_name)